## Waterlevel stations
Convert data from http://matroos.deltares.nl/timeseries/start/ (noos output) into a format that can be used in Google Fusiontable
* Terschelling Noordzee (53.44248608°N, 005.33303479°E)
* Delfzijl (53.32635727°E,6.93312255°N)
* Haringvliet 10 (51.86313430°N, 3.86077225°E)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [14]:
stations = ['Terschelling','Delfzijl','Haringvliet']

for station in stations:
    print station
    df = pd.read_csv(r'D:\hagenaar\Documents\EMODNET\GLOSSISgrid\ObservedWaterlevel' + str(station) + '.txt', 
                     skiprows=13, delimiter='  ', names=['time','waterlevel'])
    
    df['time'] = pd.to_datetime(df['time'], yearfirst=True, format='%Y%m%d%H%M')
    df.set_index('time', inplace=True)
    df['waterlevel'][df['waterlevel'].isnull()] = np.nan
    df['waterlevel'][df['waterlevel'] == 9999999] # missing value
    df.dropna(inplace=True)
    df.to_csv(r'D:\hagenaar\Documents\EMODNET\GLOSSISgrid\ObservedWaterlevel' + str(station) + 'MODIFIED.txt')

Terschelling


D:\hagenaar\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Delfzijl
Haringvliet
